import sys
!{sys.executable} -m pip install snowflake-connector-Python
!{sys.executable} -m pip install snowflake-sqlalchemy

### PLEASE CHECK IF YOU HAVEN'T ALREADY DONE THIS: For retrieving sql queries into python

import sys
!{sys.executable} -m pip install 'snowflake-connector-Python[pandas]'

!{sys.executable} -m pip install --upgrade pyarrow (Not necessary but in case you want the latest version)

In [101]:
# !{sys.executable} -m pip list

Package                    Version
-------------------------- --------------------
absl-py                    0.12.0
aiohttp                    3.7.4.post0
alembic                    1.5.8
altair                     4.1.0
anyio                      2.2.0
appdirs                    1.4.3
argon2-cffi                20.1.0
asn1crypto                 1.4.0
astroid                    2.5.6
astunparse                 1.6.3
async-generator            1.10
async-timeout              3.0.1
attrs                      20.3.0
Automat                    20.2.0
autopep8                   1.5.6
backcall                   0.2.0
bash_kernel                0.7.2
beautifulsoup4             4.9.3
black                      21.4b2
bleach                     3.3.0
blis                       0.7.4
cachetools                 4.2.2
catalogue                  2.0.4
certifi                    2019.11.28
certipy                    0.1.3
cffi                       1.14.5
chardet                    3.0.4
charset-no

In [1]:
import snowflake.connector as sf

In [125]:
Account = 'DSA57842'
UserName = 'tmxiang'
password = '9r*WWEL&9mmEzvW'

In [3]:
sfPswd = ''
# Request user password if not provided already    
if sfPswd == '' :
  import getpass
  sfPswd = getpass.getpass('Password:')

Password: ···············


In [7]:
# Test the connection to Snowflake by retrieving the version number
try:
    sfConnection = sf.connect(
        user=UserName,
        password=sfPswd,
        account=Account
    )
    sfq = sfConnection.cursor()
    sfq.execute("SELECT current_version()")
    sfResults = sfq.fetchall()
    print('Snowflake Version: ' + sfResults[0][0])
    sfq.close()
    sfConnection.close()
except:
    print('Connection failed. Check credentials')

Snowflake Version: 6.3.1


In [8]:
# Open connection to Snowflake
sfConnection = sf.connect(
    user=UserName,
    password=sfPswd,
    account=Account
)
sfq = sfConnection.cursor()

In [9]:
sfq.execute('DROP DATABASE IF EXISTS DEMO_DB')

In [10]:
sfq.execute('CREATE DATABASE DEMO_DB')
sfq.execute('USE DATABASE DEMO_DB')
sfq.execute('CREATE SCHEMA DEMO_SCHEMA')
sfq.execute('USE DEMO_DB.DEMO_SCHEMA')
sfq.execute('CREATE STAGE DEMO_STAGE')

In [11]:
sfq.execute('DROP DATABASE IF EXISTS DEMO_DB')

In [12]:
sfDatabase = 'DEMO_DB'
sfSchema = 'DEMO_SCHEMA'
sfStage = 'DEMO_STAGE'
sfq.execute('CREATE DATABASE {0}'.format(sfDatabase))
sfq.execute('USE DATABASE {0}'.format(sfDatabase))
sfq.execute('CREATE SCHEMA {0}'.format(sfSchema))
sfq.execute('USE {0}.{1}'.format(sfDatabase, sfSchema))
sfq.execute('CREATE STAGE {0}'.format(sfStage))

In [13]:
sfq.execute('DROP DATABASE IF EXISTS DEMO_DB')

In [14]:
def CreateSnowflakeDBandSchema (
    sfPswd = '',
    sfAccount = Account,
    sfUser = UserName,
    sfDatabase = 'DEMO_DB',
    sfSchema = 'DEMO_SCHEMA'
):
    import snowflake.connector as sf
    # Request user password if not provided already
    if sfPswd == '' :
      import getpass
      sfPswd = getpass.getpass('Password:')
    # Test the connection to Snowflake
    try:
      sfConnection = sf.connect(
          user=sfUser,
          password=sfPswd,
          account=sfAccount
      )
      sfq = sfConnection.cursor()
      # sfq.execute("SELECT current_version()")
      # sfResults = sfq.fetchall()
      # print('Snowflake Version: ' + sfResults[0][0])
      sfq.close()
      sfConnection.close()
    except:
      print('Connection failed. Check credentials')
    # Open connection to Snowflake
    sfConnection = sf.connect(
      user=sfUser,
      password=sfPswd,
      account=sfAccount
    )
    sfq = sfConnection.cursor()
    sfq.execute('CREATE DATABASE IF NOT EXISTS {0}'.format(sfDatabase))
    sfq.execute('CREATE SCHEMA IF NOT EXISTS {0}.{1}'.format(sfDatabase, sfSchema))
    print('Steps complete')

In [15]:
CreateSnowflakeDBandSchema (
    sfAccount = Account,
    sfUser = UserName,
    sfDatabase = 'STILL_ANOTHER_DEMO_DB',
    sfSchema = 'STILL_ANOTHER_DEMO_SCHEMA'
)

Password: ···············


Steps complete


## We need to move the CSV Files into Snowflake Stage

In [84]:
import os
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [5]:
def ConnectSnowflake (
    sfPswd = '',
    sfAccount = Account,
    sfUser = UserName,
    sfDatabase = 'MSBA_ETL',
    sfSchema = 'MSBA_ETL'
):
    import snowflake.connector as sf
    # Request user password if not provided already
    if sfPswd == '' :
      import getpass
      sfPswd = getpass.getpass('Password:')
    # Test the connection to Snowflake
    try:
      sfConnection = sf.connect(
          user=sfUser,
          password=sfPswd,
          account=sfAccount
      )
      sfq = sfConnection.cursor()
      # sfq.execute("SELECT current_version()")
      # sfResults = sfq.fetchall()
      # print('Snowflake Version: ' + sfResults[0][0])
      sfq.close()
      sfConnection.close()
    except:
      print('Connection failed. Check credentials')
    # Open connection to Snowflake
    sfConnection = sf.connect(
      user=sfUser,
      password=sfPswd,
      account=sfAccount
    )
    sfq = sfConnection.cursor()
    print('Connection opened, here is your cursor')
    return sfq

In [18]:
CreateSnowflakeDBandSchema (
    sfAccount = Account,
    sfUser = UserName,
    sfDatabase = 'MSBA_ETL',
    sfSchema = 'MSBA_ETL'
)

Password: ···············


Steps complete


In [170]:
engine = ConnectSnowflake(sfPswd=password)

Connection opened, here is your cursor


In [20]:
engine.execute('CREATE STAGE IF NOT EXISTS "MSBA_ETL"."PUBLIC".CSV_STAGE')
##engine.CREATE STAGE "DEMO_DB2"."PUBLIC".CSV_STAGE COMMENT = 'Place to stage the CSV';

In [7]:
file_name = "/data/MonthlyPOData/2013-1.csv"
file_path = os.path.relpath(file_name)
file_path

'../../../../data/MonthlyPOData/2013-1.csv'

## This doesn't work Next thing we need is to move the csv files into stage... 

Prof wants us to to do it this way.  I was just attempting to load a 
https://docs.snowflake.com/en/user-guide/data-load-considerations-stage.html


In [24]:
#engine.execute(f"put file://{file_path}* @%2013-1.csv")

ProgrammingError: 090105 (22000): Cannot perform STAGE PUT. This session does not have a current database. Call 'USE DATABASE', or use a qualified name.

## Cell below this is just to see the list of columns from the files 

In [10]:
data_sample = pd.read_csv(f"Monthly_PO_Data/{files[0]}")
data_sample.columns

Index(['PurchaseOrderID', 'SupplierID', 'OrderDate', 'DeliveryMethodID',
       'ContactPersonID', 'ExpectedDeliveryDate', 'SupplierReference',
       'IsOrderFinalized', 'Comments', 'InternalComments', 'LastEditedBy',
       'LastEditedWhen', 'PurchaseOrderLineID', 'StockItemID', 'OrderedOuters',
       'Description', 'ReceivedOuters', 'PackageTypeID',
       'ExpectedUnitPricePerOuter', 'LastReceiptDate', 'IsOrderLineFinalized',
       'Right_LastEditedBy', 'Right_LastEditedWhen'],
      dtype='object')

In [172]:
# Seting environment to creating test table: using the MSBA_ETL database and PUBLIC scheme
engine.execute("USE WAREHOUSE COMPUTE_WH")
#engine.execute("USE DATABASE MSBA_ETL")
engine.execute("USE SCHEMA MSBA_ETL.PUBLIC")

In [8]:
# Whatever folder name you guys stored your csv files in
folder = "Monthly_PO_Data/"
files = os.listdir(folder)

In [32]:
# Get the years for the files
years = set([y.split("-")[0] for y in files])
years

{'2013', '2014', '2015', '2016'}

In [220]:
len(years)

4

## ACTUAL: Staging and Copying files into official monthly_po table

In [233]:
# Identify the location of the table you want to stage and load data into
warehouse = 'COMPUTE_WH'
schema = 'MSBA_ETL.PUBLIC'

In [253]:
# user-defined functions for staging and removing files
def staging(file_pattern_lst, table, wh=warehouse, schema=schema, sub_folder=None):
    engine.execute(f"USE WAREHOUSE {wh}")
    engine.execute(f"USE SCHEMA {schema}")
    
    if (type(file_pattern_lst) == str) & (sub_folder==None):
        engine.execute(f"put file://{f}* @%{table}")
    
    else:
        for f in file_pattern_lst:
            engine.execute(f"put file://{sub_folder}/{f}* @%{table}")

        
def remove_files(staging, file_ext=None):
    if file_ext==None:
        engine.execute(f"REMOVE @{staging}")
    else:
        engine.execute(f"REMOVE @%{staging} PATTERN='.*.{file_ext}'")

In [63]:
# Create File Format if it doesn't exist already
engine.execute("CREATE FILE FORMAT IF NOT EXISTS CSV_FORMAT "
                "TYPE = 'CSV' "
                "FIELD_DELIMITER = ',' "
                "SKIP_HEADER = 1 "
                "FIELD_OPTIONALLY_ENCLOSED_BY = '\042' "
                "ERROR_ON_COLUMN_COUNT_MISMATCH = TRUE "
                "NULL_IF=('null')"
              )

In [47]:
# ... manual input to create default table and selecting the right columns to copy into table (A terrible nightmare)
engine.execute("CREATE OR REPLACE TABLE "
               "MONTHLY_PO(PurchaseOrderID number, SupplierID number, OrderDate date, DeliveryMethodID number, "
               "ExpectedDeliveryDate date, SupplierReference varchar, LastEditedBy number, PurchaseOrderLineID number, "
               "StockItemID number, OrderedOuters number, Description varchar, ReceivedOuters number, PackageTypeID number, "
               "ExpectedUnitPricePerOuter number, LastReceiptDate date, Right_LastEditedBy number)")

In [256]:
# stagging all PO files into table
staging(years, table="monthly_po", sub_folder="Monthly_PO_Data")

In [62]:
engine.execute(f"COPY INTO monthly_po FROM (SELECT $1, $2, $3, $4, $6, $7, $11, $13, $14, $15, $16, $17, $18, $19, $20, $22 FROM @%monthly_po) "
               "FILE_FORMAT = (FORMAT_NAME = CSV_FORMAT)")

In [59]:
# Can't really see it unless you go into Snowflakes, trying to figure out how to apply pandas to sql commands...
engine.execute("LIST @%monthly_po")

In [155]:
# squery statement
sql = "LIST @%monthly_po"

In [156]:
# Fetching sql query into python
engine.execute(sql)
data = engine.fetchall()

In [157]:
# What the retrieved data will look like in a pandas DF.
df = pd.DataFrame(data)
df.shape

(41, 4)

In [158]:
# Remove files from staging for best practice
remove_files('monthly_po', 'csv')

In [198]:
# New query for 100 rows from monthly_po data
sql = "select * from monthly_po limit 100;"

In [177]:
engine.execute(sql)
data = engine.fetchall()

In [178]:
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,558,4,2013-12-02,7,2013-12-22,293092,8,2160,77,767,"""The Gu"" red shirt XML tag t-shirt (White) XXS",767,6,84,2013-12-03,8
1,558,4,2013-12-02,7,2013-12-22,293092,8,2161,78,981,"""The Gu"" red shirt XML tag t-shirt (White) XS",981,6,84,2013-12-03,8
2,558,4,2013-12-02,7,2013-12-22,293092,8,2162,80,397,"""The Gu"" red shirt XML tag t-shirt (White) M",397,6,84,2013-12-03,8
3,558,4,2013-12-02,7,2013-12-22,293092,8,2163,86,816,"""The Gu"" red shirt XML tag t-shirt (White) 5XL",816,6,96,2013-12-03,8
4,558,4,2013-12-02,7,2013-12-22,293092,8,2164,95,521,"""The Gu"" red shirt XML tag t-shirt (Black) XL",521,6,90,2013-12-03,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,582,7,2013-12-17,2,2014-01-06,BC0280982,15,2255,193,408,Black and orange glass with care despatch tape...,408,7,38,2013-12-18,15
96,582,7,2013-12-17,2,2014-01-06,BC0280982,15,2256,204,483,Tape dispenser (Red),483,7,170,2013-12-18,15
97,583,4,2013-12-18,7,2014-01-07,293092,20,2257,77,837,"""The Gu"" red shirt XML tag t-shirt (White) XXS",837,6,84,2013-12-19,20
98,583,4,2013-12-18,7,2014-01-07,293092,20,2258,78,969,"""The Gu"" red shirt XML tag t-shirt (White) XS",969,6,84,2013-12-19,20


In [179]:
# close the cursor when you're done with your querying session.
engine.close()

True

### So, the cell above just checked if all 41 csv files has been uploaded into the staging table. Which is true, so moving on...

For the following part in loading the XML file, I redefined the connection function to only return the connection, not the cursor object. The cursor object will be created separately so that I can use the connection/engine for retrieve queries into pandas, the cursor for when I need to execute queries in Snowflakes

In [207]:
def ConnectSnowflake (
    sfPswd = '',
    sfAccount = Account,
    sfUser = UserName,
    sfDatabase = 'MSBA_ETL',
    sfSchema = 'PUBLIC'
):
    import snowflake.connector as sf
    # Request user password if not provided already
    if sfPswd == '' :
      import getpass
      sfPswd = getpass.getpass('Password:')
    # Test the connection to Snowflake
    try:
      sfConnection = sf.connect(
          user=sfUser,
          password=sfPswd,
          account=sfAccount
      )
      sfq = sfConnection.cursor()
      # sfq.execute("SELECT current_version()")
      # sfResults = sfq.fetchall()
      # print('Snowflake Version: ' + sfResults[0][0])
      sfq.close()
      sfConnection.close()
    except:
      print('Connection failed. Check credentials')
    # Open connection to Snowflake
    sfConnection = sf.connect(
      user=sfUser,
      password=sfPswd,
      account=sfAccount,
      warehouse="COMPUTE_WH",
      database="MSBA_ETL",
      schema="PUBLIC"
    )
    #sfq = sfConnection.cursor()
    print('Connection opened, here is your engine')
    return sfConnection

In [208]:
engine = ConnectSnowflake(sfPswd=password)

Connection opened, here is your engine


In [214]:
# read_sql_query gives me the columns too.

df = pd.read_sql_query(sql, engine)

In [216]:
type(df)

pandas.core.frame.DataFrame

In [217]:
# getting cursor
cur = engine.cursor()

In [295]:
# Creating new file format for XML if needed?
creat_xml_format = """
CREATE FILE FORMAT IF NOT EXISTS XML_FORMAT
TYPE = 'XML'
STRIP_OUTER_ELEMENT = TRUE;
"""

create_xml_table = """
CREATE OR REPLACE TABLE raw_transactions(xml_data VARIANT);"""

In [296]:
cur.execute(create_xml_table)

In [268]:
cur.execute(creat_xml_format)

In [255]:
def staging(file_pattern_lst, table, wh=warehouse, schema=schema, sub_folder=None):
    cur.execute(f"USE WAREHOUSE {wh}")
    cur.execute(f"USE SCHEMA {schema}")
    
    if (type(file_pattern_lst) == str) & (sub_folder==None):
        cur.execute(f"put file://{file_pattern_lst}* @%{table}")
    
    else:
        for f in file_pattern_lst:
            cur.execute(f"put file://{sub_folder}/{f}* @%{table}")

In [297]:
#cur.execute(f"put file://Supplier* @%raw_transactions")
staging("Supplier", table='raw_transactions')

#pd.read_sql_query("LIST @%raw_transactions", engine) # <----- checks if it's staged

In [298]:
cur.execute("""COPY INTO raw_transactions FROM @%raw_transactions FILE_FORMAT = (FORMAT_NAME = XML_FORMAT)""")

In [299]:
cur.execute("REMOVE @%raw_transactions PATTERN='.*.xml.gz'")

# pd.read_sql_query("LIST @%raw_transactions", engine) # <----- checks if it's deleted

In [300]:
sql = "select * from raw_transactions"
pd.read_sql_query(sql, engine)

# so... this calls for "flattening" or pretty much just create a view that returns a better looking table

,XML_DATA
0,<row>\n <SupplierTransactionID>134</SupplierT...
1,<row>\n <SupplierTransactionID>169</SupplierT...
2,<row>\n <SupplierTransactionID>186</SupplierT...
3,<row>\n <SupplierTransactionID>215</SupplierT...
4,<row>\n <SupplierTransactionID>224</SupplierT...
...,...
2433,<row>\n <SupplierTransactionID>335510</Suppli...
2434,<row>\n <SupplierTransactionID>335511</Suppli...
2435,<row>\n <SupplierTransactionID>335512</Suppli...
2436,<row>\n <SupplierTransactionID>335843</Suppli...


In [302]:
# I tried casting to different data types, but there's "" values in some of the columns. I have not figured out how to turn those into null values yet...
#...without trying to add a CASE WHEN to each... and I quite honestly don't wanna spend anymore on this issue.
# So, just remember to cast to the right type later on
sql = """
CREATE OR REPLACE VIEW supplier_trans_view AS(
SELECT GET(XMLGET(xml_data, 'SupplierTransactionID'), '$') AS SupplierTransactionID,
GET(XMLGET(xml_data, 'SupplierID'), '$') AS SupplierID,
GET(XMLGET(xml_data, 'TransactionTypeID'), '$') AS TransactionTypeID,
GET(XMLGET(xml_data, 'PurchaseOrderID'), '$') AS PurchaseOrderID,
GET(XMLGET(xml_data, 'PaymentMethodID'), '$') AS PaymentMethodID,
GET(XMLGET(xml_data, 'SupplierInvoiceNumber'), '$') AS SupplierInvoiceNumber,
GET(XMLGET(xml_data, 'TransactionDate'), '$') AS TransactionDate,
GET(XMLGET(xml_data, 'AmountExcludingTax'), '$') AS AmountExcludingTax,
GET(XMLGET(xml_data, 'TaxAmount'), '$') AS TaxAmount,
GET(XMLGET(xml_data, 'TransactionAmount'), '$') AS TransactionAmount,
GET(XMLGET(xml_data, 'OutstandingBalance'), '$') AS OutstandingBalance,
GET(XMLGET(xml_data, 'FinalizationDate'), '$') AS FinalizationDate,
GET(XMLGET(xml_data, 'IsFinalized'), '$') AS IsFinalized,
GET(XMLGET(xml_data, 'LastEditedBy'), '$') AS LastEditedBy
FROM raw_transactions);
"""

In [303]:
pd.read_sql_query(sql, engine)

,status
0,View SUPPLIER_TRANSACTIONS_VIEW successfully c...


In [306]:
sql = """select * from supplier_trans_view limit 200"""

In [307]:
pd.read_sql_query(sql, engine)

,SUPPLIERTRANSACTIONID,SUPPLIERID,TRANSACTIONTYPEID,PURCHASEORDERID,PAYMENTMETHODID,SUPPLIERINVOICENUMBER,TRANSACTIONDATE,AMOUNTEXCLUDINGTAX,TAXAMOUNT,TRANSACTIONAMOUNT,OUTSTANDINGBALANCE,FINALIZATIONDATE,ISFINALIZED,LASTEDITEDBY
0,134,2,5,1,4,7290,"""2013-01-02""",313.5,47.03,360.53,0,"""2013-01-07""",1,4
1,169,4,5,2,4,3898,"""2013-01-02""",21732,3259.8,24991.8,0,"""2013-01-07""",1,4
2,186,5,5,3,4,616,"""2013-01-02""",2740.5,411.11,3151.61,0,"""2013-01-07""",1,4
3,215,7,5,4,4,3869,"""2013-01-02""",42481.2,6372.19,48853.39,0,"""2013-01-07""",1,4
4,224,10,5,5,4,4697,"""2013-01-02""",35067.5,5260.14,40327.64,0,"""2013-01-07""",1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,22934,7,5,162,4,1691,"""2013-04-08""",12109.2,1816.38,13925.58,0,"""2013-04-08""",1,2
196,22937,4,5,163,4,743,"""2013-04-08""",912,136.8,1048.8,0,"""2013-04-08""",1,2
197,22939,7,5,164,4,4720,"""2013-04-08""",1020,153,1173,0,"""2013-04-08""",1,2
198,22940,4,7,"""""",4,"""""","""2013-04-08""",0,0,-433499.4,0,"""2013-04-08""",1,2


In [308]:
engine.close()

## That one supplier_case file in postgres... there's a reason I enjoyed this part

In [312]:
import psycopg2

import pandas as pd

conn = psycopg2.connect( host="127.0.0.1", port="8765",database="WestCoastImporters", user="jovyan", password="postgres")

In [314]:
# Under the assumption that you have already created the table in postgresql
sql = ("""SELECT * FROM supplier_case""")

df = pd.read_sql_query(sql, conn)

pd.DataFrame(df)

,supplierid,suppliername,suppliercategoryid,primarycontactpersonid,alternatecontactpersonid,deliverymethodid,postalcityid,supplierreference,bankaccountname,bankaccountbranch,...,deliveryaddressline1,deliveryaddressline2,deliverypostalcode,deliverylocation,postaladdressline1,postaladdressline2,postalpostalcode,lasteditedby,validfrom,validto
0,1,A Datum Corporation,2,21,22,7.0,22202,AA20384,A Datum Corporation,Woodgrove Bank Zionsville,...,Suite 10,183838 Southwest Boulevard,22202,0xE6100000010CDE115F37B6F9434031276893C39055C0,PO Box 1039,Arlington,22202,1,05:00.0,##################
1,2,"Contoso, Ltd.",2,23,24,9.0,80125,B2084020,Contoso Ltd,Woodgrove Bank Greenbank,...,Unit 2,2934 Night Road,80125,0xE6100000010CDA4B6430900C4840C04EFBF7AAA45EC0,PO Box 1012,Highlands Ranch,80125,1,05:00.0,##################
2,3,Consolidated Messenger,6,25,26,NaN,60523,209340283,Consolidated Messenger,Woodgrove Bank San Francisco,...,None,894 Market Day Street,60523,0xE6100000010C529ACDE330E34240DFFB1BB4D79A5EC0,PO Box 1014,Westmont,60523,1,05:00.0,##################
3,4,"Fabrikam, Inc.",4,27,28,7.0,95642,293092,Fabrikam Inc,Woodgrove Bank Lakeview Heights,...,Level 2,393999 Woodberg Road,95642,0xE6100000010C86E7A5626313434023C9625147E054C0,PO Box 301,Jackson,95642,1,05:00.0,##################
4,5,Graphic Design Institute,2,29,30,10.0,80125,8803922,Graphic Design Institute,Woodgrove Bank Lanagan,...,None,45th Street,80125,0xE6100000010C15E46723D74D424081F8AF62A79C57C0,PO Box 393,Highlands Ranch,80125,1,05:00.0,##################
5,6,Humongous Insurance,9,31,32,NaN,42437,82420938,Humongous Insurance,Woodgrove Bank Lancing,...,None,9893 Mount Norris Road,42437,0xE6100000010CCCF2D0D2700F424085C7235DD82955C0,PO Box 94829,Boxville,42437,1,05:00.0,##################
6,7,"Litware, Inc.",5,33,34,2.0,95642,BC0280982,Litware Inc,Woodgrove Bank Mokelumne Hill,...,Level 3,19 Le Church Street,95642,0xE6100000010C297398D475264340B63CC560342D5EC0,PO Box 20290,Jackson,95642,1,05:00.0,##################
7,8,Lucerne Publishing,2,35,36,10.0,60523,JQ082304802,Lucerne Publishing,Woodgrove Bank Jonesborough,...,Suite 34,949482 Miller Boulevard,60523,0xE6100000010C9D8F21B6AA25424091F69A794D9E54C0,PO Box 8747,Westmont,60523,1,05:00.0,##################
8,9,Nod Publishers,2,37,38,10.0,29625,GL08029802,Nod Publishers,Woodgrove Bank Elizabeth City,...,Level 1,389 King Street,29625,0xE6100000010C0EB0A07AB52542407452A923111053C0,PO Box 3390,Anderson,29625,1,05:00.0,##################
9,10,Northwind Electric Cars,3,39,40,8.0,22202,ML0300202,Northwind Electric Cars,Woodgrove Bank Crandon Lakes,...,None,440 New Road,22202,0xE6100000010C6C4E14D7E78F4440C74ED3C2C0B552C0,PO Box 30920,Arlington,22202,1,05:00.0,##################


In [315]:
conn.close()

# EVERYTHING BELOW: Code Snippets

## Below are tests (WARNING: Some cells below will not run)

In [17]:
# Creates default file format for CSV files if not exists
engine.execute("CREATE FILE FORMAT IF NOT EXISTS CSV_FORMAT "
                "TYPE = 'CSV' "
                "FIELD_DELIMITER = ',' "
                "SKIP_HEADER = 1 "
                "FIELD_OPTIONALLY_ENCLOSED_BY = '\042' "
                "ERROR_ON_COLUMN_COUNT_MISMATCH = TRUE "
              )

In [40]:
# create test table
engine.execute(
        "CREATE OR REPLACE TABLE "
        "test_table(col1 integer, col2 string)")

In [18]:
# stagging into table
engine.execute(f"put file://Monthly_PO_Data/2013* @%test_table")

In [22]:
# ... manual input to create default table and selecting the right columns to copy into table (A terrible nightmare)
engine.execute("CREATE OR REPLACE TABLE "
               "TEST_MONTHLY_PO(PurchaseOrderID number, SupplierID number, OrderDate date, DeliveryMethodID number, "
               "ExpectedDeliveryDate date, SupplierReference varchar, LastEditedBy number, PurchaseOrderLineID number, "
               "StockItemID number, OrderedOuters number, Description varchar, ReceivedOuters number, PackageTypeID number, "
               "ExpectedUnitPricePerOuter number, LastReceiptDate date, Right_LastEditedBy number)")

In [26]:
# stagging into table
engine.execute(f"put file://Monthly_PO_Data/2014* @%test_monthly_po")

In [27]:
engine.execute(f"COPY INTO test_monthly_po FROM (SELECT $1, $2, $3, $4, $6, $7, $11, $13, $14, $15, $16, $17, $18, $19, $20, $22 FROM @%test_monthly_po) "
               "FILE_FORMAT = (FORMAT_NAME = TEST_CSV_FORMAT)")

In [35]:
# Check Table
sql = "SELECT * FROM test_monthly_po"

In [36]:
pd.read_sql_query(sql, engine)

AttributeError: 'SnowflakeCursor' object has no attribute 'cursor'

In [25]:
# Retrieve list of files in stage
engine.execute("LIST @%test_monthly_po")

In [34]:
# Removing files from stage after loading is successful
engine.execute("REMOVE @%test_monthly_po PATTERN='.*.csv'")

In [25]:
# Verify that they're all removed from stage
engine.execute("LIST @%test_monthly_po")

In [159]:
# Staging into a named stage instead of a table stage...
# Depends on preference, you can either do a table stage or named stage
# But for the actual loading run, I'mma use table stage
engine.execute(f"put file://Monthly_PO_Data/2014* @CSV_STAGE")

In [168]:
# Queries a list of files. See Snowflake history
engine.execute("LIST @CSV_STAGE")

In [161]:
# Loading into table
engine.execute(f"COPY INTO test_monthly_po FROM (SELECT $1, $2, $3, $4, $6, $7, $11, $13, $14, $15, $16, $17, $18, $19, $20, $22 FROM @CSV_STAGE) "
               "FILE_FORMAT = (FORMAT_NAME = TEST_CSV_FORMAT)")

In [167]:
# Remove files from stage after loading into table
engine.execute("REMOVE @CSV_STAGE")

In [ ]:
# To get that fetch_pandas_all() method that doesn't seem to work for me yet... but it's supposed to retrieve column names as well, unlike fetchall()
# NOTE: it was also mentioned at the very top of this file, but in case you haven't done it...
import sys
!{sys.executable} -m pip install 'snowflake-connector-Python[pandas]'

In [184]:
import pandas as pd
import snowflake.connector

conn = snowflake.connector.connect(
            user=UserName,
            password="9r*WWEL&9mmEzvW",
            account=Account,
            warehouse=warehouse,
            database="MSBA_ETL",
            schema="PUBLIC"
            )

# cur = conn.cursor()

In [187]:
# defining the connecting only

df = pd.read_sql(sql, conn)

In [188]:
df

,PURCHASEORDERID,SUPPLIERID,ORDERDATE,DELIVERYMETHODID,EXPECTEDDELIVERYDATE,SUPPLIERREFERENCE,LASTEDITEDBY,PURCHASEORDERLINEID,STOCKITEMID,ORDEREDOUTERS,DESCRIPTION,RECEIVEDOUTERS,PACKAGETYPEID,EXPECTEDUNITPRICEPEROUTER,LASTRECEIPTDATE,RIGHT_LASTEDITEDBY
0,558,4,2013-12-02,7,2013-12-22,293092,8,2160,77,767,"""The Gu"" red shirt XML tag t-shirt (White) XXS",767,6,84,2013-12-03,8
1,558,4,2013-12-02,7,2013-12-22,293092,8,2161,78,981,"""The Gu"" red shirt XML tag t-shirt (White) XS",981,6,84,2013-12-03,8
2,558,4,2013-12-02,7,2013-12-22,293092,8,2162,80,397,"""The Gu"" red shirt XML tag t-shirt (White) M",397,6,84,2013-12-03,8
3,558,4,2013-12-02,7,2013-12-22,293092,8,2163,86,816,"""The Gu"" red shirt XML tag t-shirt (White) 5XL",816,6,96,2013-12-03,8
4,558,4,2013-12-02,7,2013-12-22,293092,8,2164,95,521,"""The Gu"" red shirt XML tag t-shirt (Black) XL",521,6,90,2013-12-03,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,582,7,2013-12-17,2,2014-01-06,BC0280982,15,2255,193,408,Black and orange glass with care despatch tape...,408,7,38,2013-12-18,15
96,582,7,2013-12-17,2,2014-01-06,BC0280982,15,2256,204,483,Tape dispenser (Red),483,7,170,2013-12-18,15
97,583,4,2013-12-18,7,2014-01-07,293092,20,2257,77,837,"""The Gu"" red shirt XML tag t-shirt (White) XXS",837,6,84,2013-12-19,20
98,583,4,2013-12-18,7,2014-01-07,293092,20,2258,78,969,"""The Gu"" red shirt XML tag t-shirt (White) XS",969,6,84,2013-12-19,20


In [189]:
conn.close()

In [181]:
# using cursor
cur.execute(sql)

In [132]:
df = cur.fetchall()

In [134]:
pd.DataFrame(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,558,4,2013-12-02,7,2013-12-22,293092,8,2160,77,767,"""The Gu"" red shirt XML tag t-shirt (White) XXS",767,6,84,2013-12-03,8
1,558,4,2013-12-02,7,2013-12-22,293092,8,2161,78,981,"""The Gu"" red shirt XML tag t-shirt (White) XS",981,6,84,2013-12-03,8
2,558,4,2013-12-02,7,2013-12-22,293092,8,2162,80,397,"""The Gu"" red shirt XML tag t-shirt (White) M",397,6,84,2013-12-03,8
3,558,4,2013-12-02,7,2013-12-22,293092,8,2163,86,816,"""The Gu"" red shirt XML tag t-shirt (White) 5XL",816,6,96,2013-12-03,8
4,558,4,2013-12-02,7,2013-12-22,293092,8,2164,95,521,"""The Gu"" red shirt XML tag t-shirt (Black) XL",521,6,90,2013-12-03,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,582,7,2013-12-17,2,2014-01-06,BC0280982,15,2255,193,408,Black and orange glass with care despatch tape...,408,7,38,2013-12-18,15
96,582,7,2013-12-17,2,2014-01-06,BC0280982,15,2256,204,483,Tape dispenser (Red),483,7,170,2013-12-18,15
97,583,4,2013-12-18,7,2014-01-07,293092,20,2257,77,837,"""The Gu"" red shirt XML tag t-shirt (White) XXS",837,6,84,2013-12-19,20
98,583,4,2013-12-18,7,2014-01-07,293092,20,2258,78,969,"""The Gu"" red shirt XML tag t-shirt (White) XS",969,6,84,2013-12-19,20


In [135]:
df = cur.fetch_pandas_all() # I can't figure out why it cann't find the method when it did install... but there's always the fetchall() method

ProgrammingError: 255002: Optional dependency: 'pyarrow' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation

In [136]:
sql = "LIST @%monthly_po"
cur.execute(sql)
result = cur.fetchall()

In [140]:
pd.DataFrame(result).shape

(41, 4)

In [145]:
result = cur.fetch_pandas_all()

ProgrammingError: 255002: Optional dependency: 'pyarrow' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation

In [190]:
cur.close()

False